## ReMKiT1D input generator - 0D hydrogen CRM using CRM modelbound data and term generator functionality with kinetic electrons

This notebook corresponds to the kinetic test in Section 6.3. of the ReMKiT1D code paper. It tests particle and energy conservation properties of SOL-KiT-like implementations of electron-neutral Boltzmann collisions from common_models.py

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.crm_support as crm # This is the main python module for crm construction

### Some useful constants

In [ ]:
heavySpeciesMass = 2.014 #in amus
hPlanck = 6.62607004e-34
elMass =  9.10938e-31
elCharge = 1.60218e-19


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Filepaths

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_kin_crm_test/"
rk.setHDF5Path(hdf5Filepath) 

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(5.0)
rk.setNormRefZ(1.0)

# for convenience
tempNorm = rk.normalization["eVTemperature"] 
densNorm = rk.normalization["density"]

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
dv0 = 0.01
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0 
gridObj = Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True ,interpretVGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Species initialization 

NOTE: The CRM density evolution generator assumes that all species it evolves have a density as their first associated variable and that it is an implicit and fluid variable. Also, the prebuilt Janev data requires that neutral IDs correspond to principle quantum numbers of excited states.

In [ ]:
electronSpecies = sc.Species("e",0,associatedVars=["ne"]) 
ionSpecies = sc.Species("D+",-1,atomicA=heavySpeciesMass,charge=1.0,associatedVars=["ni"])

rk.addSpecies("e",0,associatedVars=["ne"])
rk.addSpecies("D+",-1,atomicA=heavySpeciesMass,charge=1.0,associatedVars=["ni"])

numNeutrals=20
neutralDensList = ["n"+str(i) for i in range(1,numNeutrals+1)] # List of neutral density names

for neutral in neutralDensList:
    rk.addSpecies("D"+neutral[1:],int(neutral[1:]),heavySpeciesMass,associatedVars=[neutral])

# Add electron temperature derivation
rk.setStandardTextbookOptions([0]) 


### Variable initialization

In [ ]:
T = np.ones(gridObj.numX())

rk.addVar("time",isScalar=True,isDerived=True,outputVar=True)

n = np.ones(gridObj.numX())  
n1 = 0.1*np.ones(gridObj.numX())

f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (T[i]*np.pi)**(-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = gridObj.velocityMoment(f,0,1)
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]
    
rk.addVar("f",f,isDistribution=True)
rk.addVar("ne",n,isDerived=True,units='$10^{19} m^{-3}$',derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("ni",n,units='$10^{19} m^{-3}$',outputVar=True)

W = 3*n*T/2

rk.addVar("We",W,units='$10eV$',isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"]))
rk.addVar("zeroVar",isDerived=True,outputVar=False)
rk.addVar("Te",T,isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["We","ne","zeroVar"]))

rk.addVar("n1",n1*np.ones(gridObj.numX()),units='$10^{19} m^{-3}$',outputVar=True)
for i in range(2,numNeutrals+1):
    rk.addVar(neutralDensList[i-1],units='$10^{19} m^{-3}$',outputVar=True)

### External libraries

#### MPI

Single processor setup.

In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic direction
numProcs = numProcsH*numProcsX
haloWidth = 1 # Halo width in cells
rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

Defaults.

#### HDF5

Output variables set when adding variables

### Models

#### Generating modelbound data based on inbuilt cross-sections

In [ ]:
includedJanevTransitions = ["ex","deex","ion","recomb3b"] 
mbData = crm.ModelboundCRMData()
crm.addJanevTransitionsToCRMData(mbData,numNeutrals,tempNorm,"f","Te",detailedBalanceCSPriority=1,processes=includedJanevTransitions,lowestCellEnergy=gridObj.vGrid[0]**2)

#### Creating the CRM model object

In [ ]:
#CRM model
 
#Adding the model tag to tag list
modelTag = "CRMmodel"

#Initializing model
crmModel = sc.CustomModel(modelTag=modelTag)

crmModel.setModelboundData(mbData.dict())

#Add term generator responsible for buildling CRM model for ions
crmTermGeneratorIons = crm.termGeneratorCRM(evolvedSpeciesIDs=[-1])

crmModel.addTermGenerator("ionCRM",crmTermGeneratorIons)

#Add term generator responsible for buildling CRM model for all neutral states

crmTermGeneratorNeutrals = crm.termGeneratorCRM(evolvedSpeciesIDs=list(range(1,numNeutrals+1)))

crmModel.addTermGenerator("neutralCRM",crmTermGeneratorNeutrals)

#Add Boltzmann term generator for excitation
exInds,exEnergies = mbData.getTransitionIndicesAndEnergies("JanevEx")

crmBoltzTermGenExE = crm.termGeneratorCRMBoltz("f",1,exInds,exEnergies,implicitTermGroups=[1]) #Emission terms

crmModel.addTermGenerator("exCRME",crmBoltzTermGenExE)

crmBoltzTermGenExA = crm.termGeneratorCRMBoltz("f",1,exInds,exEnergies,absorptionTerms=True,implicitTermGroups=[1]) #Absorption terms

crmModel.addTermGenerator("exCRMA",crmBoltzTermGenExA)

#Add Boltzmann term generators for ionization

ionInds,ionEnergies = mbData.getTransitionIndicesAndEnergies("JanevIon")

crmBoltzTermGenIonE = crm.termGeneratorCRMBoltz("f",1,ionInds,ionEnergies) #Emission terms

crmModel.addTermGenerator("ionCRME",crmBoltzTermGenIonE)

crmBoltzTermGenIonA = crm.termGeneratorCRMBoltz("f",1,ionInds,ionEnergies,absorptionTerms=True) #Absorption terms

crmModel.addTermGenerator("ionCRMA",crmBoltzTermGenIonA)

#Add Boltzmann term generators for deexcitation

deexInds,deexEnergies = mbData.getTransitionIndicesAndEnergies("JanevDeex")

crmBoltzTermGenDeexE = crm.termGeneratorCRMBoltz("f",1,deexInds,deexEnergies,detailedBalanceTerms=True,implicitTermGroups=[2]) #Emission terms

crmModel.addTermGenerator("deexCRME",crmBoltzTermGenDeexE)

crmBoltzTermGenDeexA = crm.termGeneratorCRMBoltz("f",1,deexInds,deexEnergies,absorptionTerms=True,detailedBalanceTerms=True,implicitTermGroups=[2]) #Absorption terms

crmModel.addTermGenerator("deexCRMA",crmBoltzTermGenDeexA)

# #Add Boltzmann term generators for 3b recombination

recomb3bInds,recomb3bEnergies = mbData.getTransitionIndicesAndEnergies("JanevRecomb3b")

crmBoltzTermGen3bRecombE = crm.termGeneratorCRMBoltz("f",1,recomb3bInds,recomb3bEnergies,detailedBalanceTerms=True) #Emission terms

crmModel.addTermGenerator("recomb3bCRME",crmBoltzTermGen3bRecombE)

crmBoltzTermGen3bRecombA = crm.termGeneratorCRMBoltz("f",1,recomb3bInds,recomb3bEnergies,absorptionTerms=True,detailedBalanceTerms=True) #Absorption terms

crmModel.addTermGenerator("recomb3bCRMA",crmBoltzTermGen3bRecombA)

#Add secondary electron sources/sinks due to ionization and recombination

secElInds = ionInds + recomb3bInds 

crmSecElTermGen = crm.termGeneratorCRMSecEl("f",secElInds)

crmModel.addTermGenerator("secElCRM",crmSecElTermGen)

#Add model to wrapper

rk.addModel(crmModel.dict())


### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(convergenceVars=["f","We","n1","n2","Te"],nonlinTol=1.0e-14) 

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep = 0.5

rk.setIntegratorGlobalData(2,2,initialTimestep) 

Single integration step

In [ ]:
bdeStep = sc.IntegrationStep("BE",defaultEvaluateGroups=[1,2],defaultUpdateGroups=[1,2])

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(30000)
rk.setFixedStepOutput(1000)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 30

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore="zeroVar")
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

In [ ]:
eTot = loadedData["We"].data[:,0] + loadedData["ni"].data[:,0]*13.6/tempNorm

for i,state in enumerate(neutralDensList):
    eTot = eTot + loadedData[state].data[:,0] * 13.6 * (1 - 1/(i+1)**2)/tempNorm

dE=abs(eTot - eTot[0])/eTot[0]

ntot = loadedData["ni"].data[:,0]

for state in neutralDensList:
    ntot = ntot + loadedData[state].data[:,0]

dn =abs(ntot-ntot[0])/ntot[0]

timeNorm = 0.27699197412978324E-7

eCurve = hv.Curve([(t*timeNorm*1000,dE[i]) for i,t in enumerate(loadedData.coords["time"])],label="E").opts(linestyle="--")

nCurve = hv.Curve([(t*timeNorm*1000,dn[i]) for i,t in enumerate(loadedData.coords["time"])],label="n")

curve = nCurve*eCurve
curve.opts(xlabel="t [ms]",ylabel='$\delta$')


In [ ]:
hv.save(curve.opts(xlabel="t [ms]",ylabel='$\delta$'),"kin_crm_test.pdf",dpi=144)